<a href="https://colab.research.google.com/github/masoudcharkhabi/ML-from-Expert-Preferences/blob/abstractions/colab_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Packages

In [1]:
from google.colab import drive
import os
from getpass import getpass
import wandb
import os
import wandb
import datetime
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, AutoModelForSequenceClassification.

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow info, warning, and error messages.


### Huggingface login

In [2]:
# Prompt for Hugging Face API token without saving it
huggingface_token = getpass("Please enter your Hugging Face API token: ")

# Set the token as an environment variable
os.environ["HUGGINGFACE_TOKEN"] = huggingface_token

# Log in using the token
from huggingface_hub import login
login(token=os.environ["HUGGINGFACE_TOKEN"])

Please enter your Hugging Face API token: ··········


### Make sure you have a GPU and High memory

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Thu Dec  5 13:55:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   53C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Clone Github repo (you need private-public keys with the same naming convention)

In [4]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
# Create symbolic links to the SSH keys in Drive
!ln -s /content/drive/MyDrive/ssh_keys/id_colab ~/.ssh/id_colab
!ln -s /content/drive/MyDrive/ssh_keys/id_colab.pub ~/.ssh/id_colab.pub

# Set the correct permissions for SSH keys
!chmod 600 ~/.ssh/id_colab
!chmod 644 ~/.ssh/id_colab.pub


ln: failed to create symbolic link '/root/.ssh/id_colab': File exists
ln: failed to create symbolic link '/root/.ssh/id_colab.pub': File exists


In [17]:
# Start SSH agent and add key
!eval "$(ssh-agent -s)"
!ssh-add ~/.ssh/id_colab

# Create SSH config
ssh_config = """
Host github.com
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_colab
  StrictHostKeyChecking no
"""

# Create the .ssh directory if it doesn't exist
os.makedirs(os.path.expanduser("~/.ssh"), exist_ok=True)

with open(os.path.expanduser("~/.ssh/config"), "w") as f:
    f.write(ssh_config)

Agent pid 2874
Could not open a connection to your authentication agent.


In [18]:
# Test SSH connection
!ssh -T git@github.com

Hi masoudcharkhabi! You've successfully authenticated, but GitHub does not provide shell access.


In [19]:
repo_ssh_url = "git@github.com:masoudcharkhabi/ML-from-Expert-Preferences.git"
branch_name = "abstractions"
!git clone -b {branch_name} --single-branch {repo_ssh_url}

Cloning into 'ML-from-Expert-Preferences'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 319 (delta 37), reused 46 (delta 25), pack-reused 233 (from 1)
Receiving objects: 100% (319/319), 379.59 KiB | 1.49 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [20]:
repo_name = "ML-from-Expert-Preferences"
os.chdir(repo_name)

In [21]:
!git branch
!ls -ltr

* abstractions
total 104
-rw-r--r-- 1 root root  2680 Dec  5 13:58 README.md
-rw-r--r-- 1 root root 68772 Dec  5 13:58 colab_experiments.ipynb
drwxr-xr-x 3 root root  4096 Dec  5 13:58 baseline
drwxr-xr-x 4 root root  4096 Dec  5 13:58 cs329h-project
-rw-r--r-- 1 root root   190 Dec  5 13:58 requirements.txt
-rw-r--r-- 1 root root  3515 Dec  5 13:58 eval.py
-rw-r--r-- 1 root root  1966 Dec  5 13:58 data_prep.py
drwxr-xr-x 5 root root  4096 Dec  5 13:58 data
-rw-r--r-- 1 root root  1938 Dec  5 13:58 train.py
-rw-r--r-- 1 root root  1035 Dec  5 13:58 serve.py


In [22]:
!pip install -r requirements.txt

#TODO figure out the verions and added to requirements later
!pip install datasets
!pip install torch
!pip install evaluate
!pip install rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d9fc4af2c85fd38b76a18a2597be110520f2cd5c7c25e46d1cfca49343c03347
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

# Data prep

In [35]:
from transformers import AutoTokenizer
from data_prep import DataPreparation

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Replace "t5-small" with the appropriate model name

# Create instance of DataPreparation
data_preparation = DataPreparation(dataset_path="ai2-adapt-dev/flan_v2_converted")

# Load the dataset
dataset = data_preparation.load_data()

# Prepare the dataset (tokenize)
train_dataset = data_preparation.prepare_dataset(tokenizer=tokenizer)

# Now train_dataset is ready for training
# Get dataset information
data_preparation.dataset_info()

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train-00000-of-00001.parquet:   0%|          | 0.00/126M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/89982 [00:00<?, ? examples/s]

Map:   0%|          | 0/89982 [00:00<?, ? examples/s]

Map:   0%|          | 0/89982 [00:00<?, ? examples/s]

Split: train, Number of examples: 89982


# Fine-tune

### This will require and Weights and Biases API key for logging

In [54]:
# Import necessary classes
from train import ModelTrainer

# Step 1: Load tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Load and prepare the dataset
data_preparation = DataPreparation(dataset_path="ai2-adapt-dev/flan_v2_converted")
dataset = data_preparation.load_data()
train_dataset = data_preparation.prepare_dataset(tokenizer=tokenizer)

# Step 3: Initialize ModelTrainer and setup training
trainer = ModelTrainer(model_name=model_name)
trainer.setup_training(train_dataset=train_dataset, tokenizer=tokenizer)

# Step 4: Train the model
trainer.train_model()

Repo card metadata block was not found. Setting CardData to empty.


<ipython-input-39-43213d648f1a>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self.trainer = Trainer(


Step,Training Loss


Model saved to: ./models/fine_tuned_model_20241205_154441


train/epoch,▁
train/global_step,▁
total_flos,6225722867712.0
train/epoch,0.00051
train/global_step,23
train_loss,18.22854
train_runtime,3.6912
train_samples_per_second,12.189
train_steps_per_second,6.231


### Load the finetuned model to HF

In [56]:
# Step 5: Load to HF
# Authenticate with Hugging Face
login(token=huggingface_token)

# Define paths and names
fine_tuned_model_name = "fine_tuned_model_20241205_154441"
hf_model_name = "active-llm-winner"
repo_id = "coderGenMC/" + hf_model_name  # Ensure this matches your username

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("./models/" + fine_tuned_model_name)
model = AutoModelForSequenceClassification.from_pretrained("./models/" + fine_tuned_model_name)

# Create README content with model name
readme_content = f"""
    ----------------------------------------------------------------------------
    language: en
    tags:
    - sequence-classification
    - transformers
    - t5
    license: apache-2.0
    ---

    # {hf_model_name}

    **Model Name:** `{hf_model_name}`
    **Model Owner:** [coderGenMC](https://huggingface.co/coderGenMC)

    ## Model Description

    This model is a fine-tuned version of T5 for sequence classification tasks. It has been trained to classify text inputs into predefined categories.

    ## Training Details

    - **Model Architecture:** T5
    - **Fine-Tuned For:** Sequence Classification

    ## How to Use

    You can use this model directly with the Transformers library

    ----------------------------------------------------------------------------
    """

# Create the directory if it doesn't exist
save_directory = "./models/" + fine_tuned_model_name
os.makedirs(save_directory, exist_ok=True)

# Define the path for README.md
readme_path = os.path.join(save_directory, "README.md")

# Write the readme_content to README.md
with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme_content)

model.save_pretrained(hf_model_name)
tokenizer.push_to_hub("coderGenMC/"+hf_model_name)
model.push_to_hub("coderGenMC/"+hf_model_name, use_auth_token=huggingface_token)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at ./models/fine_tuned_model_20241205_154441 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No files have been modified since last commit. Skipping to prevent empty commit.
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/243M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/coderGenMC/active-llm-winner/commit/8f8756de418848981eb90de282123fe07ac5a80f', commit_message='Upload T5ForSequenceClassification', commit_description='', oid='8f8756de418848981eb90de282123fe07ac5a80f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/coderGenMC/active-llm-winner', endpoint='https://huggingface.co', repo_type='model', repo_id='coderGenMC/active-llm-winner'), pr_revision=None, pr_num=None)

# Serve

In [ ]:
# from serve import ModelServer

# Step 1: Load the tokenizer
model_name = "t5-small"  # TODO: Replace with fine-tuned model later
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Initialize ModelServer
model_server = ModelServer(model_name=model_name)

# Step 3: Set the tokenizer for the model server
model_server.set_tokenizer(tokenizer)

# Step 4: Run inference
input_text = "Translate the following sentence to French: 'The weather is nice today.'"
output_text = model_server.run_inference(input_text)

# Step 5: Print the result
print("Generated Output:", output_text)

# Optional: Store the output in a file
experiment_id = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = "./data/output/"+experiment_id+"_output.txt"
model_server.store_output(input_text, output_path)
print(f"Output stored in: {output_path}")


# Eval

In [ ]:
from eval import ModelEvaluator

# Step 1: Load the tokenizer
model_name = "t5-small"

# TODO: Replace with  fine-tuned model path (including the correct timestamp) later
# model_name = "./fine_tuned_model_20231124_123456"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Create instance of DataPreparation and load the dataset
data_preparation = DataPreparation(dataset_path="ai2-adapt-dev/flan_v2_converted")
dataset = data_preparation.load_data()

# Step 3: Prepare evaluation dataset
eval_dataset = data_preparation.prepare_dataset(tokenizer=tokenizer)

# Step 4: Initialize ModelEvaluator
evaluator = ModelEvaluator(model_name=model_name, tokenizer=tokenizer)

# Step 5: Run evaluation
evaluation_results = evaluator.evaluate(eval_dataset)

# Step 6: Print the evaluation results
print("Evaluation Results:", evaluation_results)
